## Example of how to discover the available Met Office NEMO and SI3 ocean and ice data

Currently only the UM atmosphere data has been CMORised and added to the intake catalogue. This Notebook shows a method of scanning through the data files to identify the available ocean variables.

Find all ocean variables available:

In [1]:
! find /gws/nopw/j04/eerie/public/data | grep '\.nc$' | grep 'nemo' | awk -F/ '{print $NF}' | awk 'BEGIN{FS="_"; OFS="_"} {print $1, $2, $3, $4, $6}' | sort -u

mlotst10_nemo_u-cx993_1d_grid-025.nc
mlotst10_nemo_u-cx993_1m_grid-025.nc
mlotst10_nemo_u-cy021_1d_grid-025.nc
mlotst10_nemo_u-cy021_1m_grid-025.nc
sosabs_nemo_u-cx993_1d_grid-025.nc
sosabs_nemo_u-cx993_1m_grid-025.nc
sosabs_nemo_u-cy021_1d_grid-025.nc
sosabs_nemo_u-cy021_1m_grid-025.nc
toscon_nemo_u-cx993_1d_grid-025.nc
toscon_nemo_u-cx993_1m_grid-025.nc
toscon_nemo_u-cy021_1d_grid-025.nc
toscon_nemo_u-cy021_1m_grid-025.nc
zos_nemo_u-cx993_1d_grid-025.nc
zos_nemo_u-cx993_1m_grid-025.nc
zos_nemo_u-cy021_1d_grid-025.nc
zos_nemo_u-cy021_1m_grid-025.nc


Find all seaice variables available:

In [2]:
! find /gws/nopw/j04/eerie/public/data | grep '\.nc$' | grep 'si3' | awk -F/ '{print $NF}' | awk 'BEGIN{FS="_"; OFS="_"} {print $1, $2, $3, $4, $6}' | sort -u

siconc_si3_u-cx993_1d_grid-025.nc
siconc_si3_u-cy021_1d_grid-025.nc
sithick_si3_u-cx993_1d_grid-025.nc
sithick_si3_u-cy021_1d_grid-025.nc
siu_si3_u-cx993_1d_grid-025.nc
siu_si3_u-cy021_1d_grid-025.nc
siv_si3_u-cx993_1d_grid-025.nc
siv_si3_u-cy021_1d_grid-025.nc


These techniques can be used to load data into Iris (and similarly Xarray):

In [3]:
import subprocess
import warnings

import iris

In [4]:
def load_callback_nemo(cube, field, filename):
    """
    An Iris load callback function to delete a few metadata items that will
    prevent the files from being merged into a single cube.
    """
    attributes_to_remove = [
        "history",
        "name",
        "timeStamp",
        "uuid"
    ]
    for attr_name in attributes_to_remove:
        del(cube.attributes[attr_name])

In [5]:
cmd_out = subprocess.run(
    "find /gws/nopw/j04/eerie/public/data | grep '\.nc$' | grep 'toscon_nemo_u-cx993_1d'", 
    shell=True, 
    capture_output=True
)

In [6]:
file_paths = [str(file_path) for file_path in cmd_out.stdout.decode("utf-8").split()]

In [7]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Just load the first three files to save time, normally the [:3] can be omitted
    cubes = iris.load(file_paths[:3], callback=load_callback_nemo)
toscon = cubes.concatenate_cube()
print(toscon.summary(shorten=True))

sea surface conservative temperature / (degC) (time: 90; latitude: 721; longitude: 1440)
